In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import requests
from scipy import stats
from scipy.stats import normaltest, skew, kurtosis

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [8]:
def get_user_animelist(username: str, client_id: str):
    # Loads list of anime from MAL for a given user
    url = f'https://api.myanimelist.net/v2/users/{username}/animelist?limit=500'
    headers = {
        'X-MAL-CLIENT-ID': client_id
    }
    params = {
        'fields': 'id, title, list_status{score,status}, start_season{year}, mean, genres, popularity, media_type, rating, num_episodes, studios, num_list_users,favorites'
    }

    all_data = []
    next_page = url

    while next_page:
        response = requests.get(next_page, headers=headers, params=params if next_page == url else None)
        if response.status_code != 200:
            raise ValueError(f"Error with API request: {response.status_code} - {response.text}")

        data = response.json()
        all_data.extend(data['data'])
        next_page = data.get("paging", {}).get("next")

        print(f"Loaded {len(all_data)} anime...")

    return all_data

In [9]:
def to_dataframe(all_data):
    rows = []
    for item in all_data:
        anime = item['node']
        score = item.get('list_status', {}).get('score')
        status = item.get('list_status', {}).get('status')
        # num_episodes_watched = item.get('list_status', {}).get('num_episodes_watched')
        year = item.get('node', {}).get('start_season', {}).get('year')
        rows.append({
            "id": anime['id'],
            "title": anime['title'],
            "mean": anime.get('mean'),
            "genres": [g['name'] for g in anime.get('genres', [])],
            "studios": [s['name'] for s in anime.get('studios', [])],
            "rating": anime.get('rating'),
            "year": year,
            "type": anime.get('media_type'),
            "popularity": anime.get('popularity'),
            "score": score,
            "status": status,
            "members": anime['num_list_users'],
            "num_episodes": anime['num_episodes'],
        })
    
    df = pd.DataFrame(rows)
    df['studios'] = df['studios'].str.join(", ")
    df['genres'] = df['genres'].str.join(", ")

    return df

In [10]:
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')
USER_NAME = config.get('USER','USER_NAME').strip()
CLIENT_ID = config.get('USER','CLIENT_ID').strip()

In [11]:
data = get_user_animelist(USER_NAME, CLIENT_ID)

In [12]:
df = to_dataframe(data)

In [29]:
df = df[df['score']>0].copy()

In [30]:
# Basic information
print("Dataset size:", df.shape)
print("\n" + "="*50)
print("FIRST ROWS:")
print(df.head(10))
print("\n" + "="*50)
print("LAST ROWS:")
print(df.tail(10))
print("\n" + "="*50)
print("RANDOM SAMPLE:")
print(df.sample(10))

In [31]:
# Data types
print("DATA TYPES INFORMATION:")
print(df.info())
print("\n" + "="*50)

# Descriptive statistics
print("DESCRIPTIVE STATISTICS:")
print(df.describe(include='all').T)
print("\n" + "="*50)

# Unique values for categorical features
print("UNIQUE VALUES:")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].nunique()} unique values")
    if df[col].nunique() < 20:
        print(f"  Values: {df[col].unique()}\n")

In [32]:
fig, axes = plt.subplots(figsize=(15, 10))
# Histogram
axes.hist(df['score'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
axes.set_title('Score Distribution', fontsize=14, fontweight='bold')
axes.set_xlabel('Score')
axes.set_ylabel('Frequency')
axes.axvline(df['score'].mean(), color='red', linestyle='--', 
                    label=f'Mean: {df["score"].mean():.2f}')
axes.axvline(df['score'].median(), color='green', linestyle='--', 
                    label=f'Median: {df["score"].median():.2f}')
axes.legend()


plt.tight_layout()
plt.show()

# Statistical tests
print("TARGET VARIABLE STATISTICS:")
print(f"Mean: {df['score'].mean():.3f}")
print(f"Median: {df['score'].median():.3f}")
print(f"Mode: {df['score'].mode().values[0]:.3f}")
print(f"Standard Deviation: {df['score'].std():.3f}")
print(f"Skewness: {skew(df['score'].dropna()):.3f}")
print(f"Kurtosis: {kurtosis(df['score'].dropna()):.3f}")
print(f"Min: {df['score'].min():.3f}")
print(f"Max: {df['score'].max():.3f}")
print(f"Range: {df['score'].max() - df['score'].min():.3f}")
print(f"\nNormality Test (D'Agostino-Pearson):")
stat, p_value = normaltest(df['score'].dropna())
print(f"  Statistic: {stat:.3f}, p-value: {p_value:.6f}")
if p_value < 0.05:
    print("  Distribution is NOT normal (p < 0.05)")
else:
    print("  Distribution is normal (p >= 0.05)")

In [34]:
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'score' in numerical_cols:
    numerical_cols.remove('score')  # Already analyzed separately

n_cols = len(numerical_cols)
n_rows = (n_cols + 2) // 3

fig, axes = plt.subplots(n_rows, 3, figsize=(18, 5*n_rows))
if n_rows == 1:
    axes = axes.reshape(1, -1)
axes = axes.flatten()

for idx, col in enumerate(numerical_cols):
    data_to_plot = df[col].dropna()
    axes[idx].hist(data_to_plot, bins=50, edgecolor='black', alpha=0.7, color='teal')
    axes[idx].set_title(f'Distribution: {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].axvline(data_to_plot.mean(), color='red', linestyle='--', 
                      label=f'Mean: {data_to_plot.mean():.2f}')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

# Hide extra subplots
for idx in range(n_cols, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [35]:
categorical_cols = ['rating', 'type', 'status']
for col in categorical_cols:
    plt.figure(figsize=(14, 6))
    
    # Frequency table
    value_counts = df[col].value_counts()
    
    # If too many categories, show top 20
    if len(value_counts) > 20:
        value_counts = value_counts.head(20)
        title_suffix = " (Top 20)"
    else:
        title_suffix = ""
    
    plt.subplot(1, 2, 1)
    value_counts.plot(kind='barh', color='steelblue')
    plt.title(f'Distribution: {col}{title_suffix}', fontsize=14, fontweight='bold')
    plt.xlabel('Count')
    plt.grid(axis='x', alpha=0.3)
    
    plt.subplot(1, 2, 2)
    value_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title(f'Proportions: {col}{title_suffix}', fontsize=14, fontweight='bold')
    plt.ylabel('')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSTATISTICS FOR {col}:")
    print(f"Unique values: {df[col].nunique()}")
    print(f"Most frequent: {df[col].mode().values[0]} ({df[col].value_counts().iloc[0]} times)")
    print(f"Least frequent: {value_counts.iloc[-1]} occurrences")

In [36]:
# Correlation of numerical features with score
correlations = df[numerical_cols + ['score']].corr()['score'].sort_values(ascending=False)

plt.figure(figsize=(10, max(8, len(correlations)*0.4)))
colors = ['green' if x > 0 else 'red' for x in correlations.drop('score')]
correlations.drop('score').plot(kind='barh', color=colors)
plt.title('Feature Correlation with Score', fontsize=16, fontweight='bold')
plt.xlabel('Pearson Correlation')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("CORRELATION WITH TARGET VARIABLE:")
print(correlations)